In [1]:
import numpy as np

In [2]:
def get_model(save_name='Xception.hdf5', image_shape=(299, 299, 3)):
    import os
    new_model = not os.path.exists(save_name)
    if new_model:
        from keras.applications.xception import Xception
        model = Xception(
            include_top=False, weights='imagenet',
            input_shape=image_shape, pooling='avg'
        )
        model.save(save_name)
    else:
        from keras.models import load_model
        model = load_model(save_name)
    return model, new_model

In [3]:
initial_model, new_model = get_model()
if new_model:
    print('new model')
else:
    print('old model')

Using TensorFlow backend.


old model


/usr/lib/python3.6/site-packages/keras/models.py:245: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
def get_data(image_shape=(299, 299, 3)):
    from skimage.transform import resize
    from skimage.io import imread
    import os
    images = []
    labels = []
    with open('public_data/00_input/train/gt.csv') as navigate_file:
        for i, line in enumerate(navigate_file):
            if i == 0:
                continue
            image_name, label = line.split(',')                
            images.append(resize(
                imread(os.path.join(
                    'public_data/00_input/train/images/',
                    image_name
                )),
                image_shape,
                mode='reflect'
            ))
            labels.append(int(label))
    return np.array(images), np.array(labels)

In [5]:
x, y = get_data()

In [7]:
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from keras.optimizers import Nadam
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [8]:
num_classes = len(set(y))
y = np.eye(num_classes)[y]
train_x, test_x, train_y, test_y = train_test_split(
    x, y, test_size=0.3, random_state=42
)

In [9]:
data_generator = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
)

In [11]:
regularization_lambda = 1e-3

last = initial_model.output

nn = Dense(
    1024, activation='elu',
    kernel_regularizer=regularizers.l2(regularization_lambda)
)(last)
nn = Dense(
    1024, activation='elu',
    kernel_regularizer=regularizers.l2(regularization_lambda)
)(nn)

prediction = Dense(
    num_classes, activation='softmax',
    kernel_regularizer=regularizers.l2(regularization_lambda)
)(nn)

model = Model(initial_model.input, prediction)

for layer in initial_model.layers:
    layer.trainable = False

In [12]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(lr=1e-3, decay=1e-3),
    metrics=['categorical_accuracy']
)

In [13]:
class MyClallback:
    def __init__(self, model, directory):
        self._model = model
        self._directory = directory
        self._counter = 1

    def set_model(self, model):
        pass

    def set_params(self, params):
        pass

    def on_train_begin(self, logs):
        pass

    def on_train_end(self, logs):
        pass

    def on_epoch_begin(self, epoch, logs):
        pass

    def on_epoch_end(self, epoch, logs):
        self._model.save(
            self._directory + '/' + str(self._counter) +
            '_val_accuracy:_' + str(logs['val_categorical_accuracy']) +
            '.npz'
        )
        self._counter += 1

    def on_batch_begin(self, batch, logs):
        pass

    def on_batch_end(self, batch, logs):
        pass

In [14]:
batch_size = 4
model.fit_generator(
    data_generator.flow(train_x, train_y, batch_size=batch_size),
    steps_per_epoch=int(len(train_y) / batch_size), epochs=30,
    callbacks=[MyClallback(model, 'untrainable_layers')],
    validation_data=(test_x, test_y)
)

Epoch 1/30
437/437 [==============================] - 114s - loss: 5.5074 - categorical_accuracy: 0.2775 - val_loss: 3.7502 - val_categorical_accuracy: 0.4747
Epoch 2/30
437/437 [==============================] - 66s - loss: 4.0411 - categorical_accuracy: 0.4273 - val_loss: 3.2689 - val_categorical_accuracy: 0.5360
Epoch 3/30
437/437 [==============================] - 66s - loss: 3.3742 - categorical_accuracy: 0.4868 - val_loss: 2.8602 - val_categorical_accuracy: 0.5813
Epoch 4/30
437/437 [==============================] - 65s - loss: 2.8308 - categorical_accuracy: 0.5543 - val_loss: 2.5409 - val_categorical_accuracy: 0.5933
Epoch 5/30
437/437 [==============================] - 66s - loss: 2.5224 - categorical_accuracy: 0.6041 - val_loss: 2.3166 - val_categorical_accuracy: 0.6280
Epoch 6/30
437/437 [==============================] - 66s - loss: 2.3521 - categorical_accuracy: 0.6024 - val_loss: 1.9981 - val_categorical_accuracy: 0.6973
Epoch 7/30
437/437 [==============================]

In [15]:
trainable_border = 107
for layer in model.layers[:trainable_border]:
    layer.trainable = False
for layer in model.layers[trainable_border:]:
    layer.trainable = True

In [16]:
batch_size = 4
model.fit_generator(
    data_generator.flow(train_x, train_y, batch_size=batch_size),
    steps_per_epoch=int(len(train_y) / batch_size), epochs=60,
    callbacks=[MyClallback(model, 'untrainable_layers')],
    validation_data=(test_x, test_y)
)

Epoch 1/60
437/437 [==============================] - 70s - loss: 1.0855 - categorical_accuracy: 0.8146 - val_loss: 1.3542 - val_categorical_accuracy: 0.7160
Epoch 2/60
437/437 [==============================] - 66s - loss: 1.0715 - categorical_accuracy: 0.8181 - val_loss: 1.3089 - val_categorical_accuracy: 0.7293
Epoch 3/60
437/437 [==============================] - 66s - loss: 1.0618 - categorical_accuracy: 0.8101 - val_loss: 1.3480 - val_categorical_accuracy: 0.7133
Epoch 4/60
437/437 [==============================] - 67s - loss: 1.0736 - categorical_accuracy: 0.8101 - val_loss: 1.3233 - val_categorical_accuracy: 0.7347
Epoch 5/60
437/437 [==============================] - 68s - loss: 1.0221 - categorical_accuracy: 0.8307 - val_loss: 1.2944 - val_categorical_accuracy: 0.7240
Epoch 6/60
437/437 [==============================] - 69s - loss: 1.0106 - categorical_accuracy: 0.8398 - val_loss: 1.3355 - val_categorical_accuracy: 0.7267
Epoch 7/60
437/437 [==============================] 

437/437 [==============================] - 68s - loss: 0.8695 - categorical_accuracy: 0.8781 - val_loss: 1.2594 - val_categorical_accuracy: 0.7267
Epoch 53/60
437/437 [==============================] - 69s - loss: 0.8843 - categorical_accuracy: 0.8656 - val_loss: 1.2506 - val_categorical_accuracy: 0.7347
Epoch 54/60
437/437 [==============================] - 68s - loss: 0.8669 - categorical_accuracy: 0.8724 - val_loss: 1.2543 - val_categorical_accuracy: 0.7360
Epoch 55/60
437/437 [==============================] - 69s - loss: 0.8887 - categorical_accuracy: 0.8616 - val_loss: 1.2554 - val_categorical_accuracy: 0.7400
Epoch 56/60
437/437 [==============================] - 69s - loss: 0.8468 - categorical_accuracy: 0.8862 - val_loss: 1.2483 - val_categorical_accuracy: 0.7453
Epoch 57/60
437/437 [==============================] - 68s - loss: 0.8605 - categorical_accuracy: 0.8667 - val_loss: 1.2367 - val_categorical_accuracy: 0.7373
Epoch 58/60
437/437 [==============================] - 68s

In [18]:
from keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(
    monitor='val_acc',
    patience=15,
    verbose=1,
    epsilon=0.006,
    min_lr=1e-8
)

In [19]:
batch_size = 4
model.fit_generator(
    data_generator.flow(train_x, train_y, batch_size=batch_size),
    steps_per_epoch=int(len(train_y) / batch_size), epochs=60,
    callbacks=[MyClallback(model, 'trainable_layers'), reduce_lr],
    validation_data=(test_x, test_y)
)

Epoch 1/60
437/437 [==============================] - 71s - loss: 0.8420 - categorical_accuracy: 0.8764 - val_loss: 1.2296 - val_categorical_accuracy: 0.7347
Epoch 2/60
  2/437 [..............................] - ETA: 35s - loss: 0.9668 - categorical_accuracy: 0.7500

/usr/lib/python3.6/site-packages/keras/callbacks.py:879: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_categorical_accuracy,loss,categorical_accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


437/437 [==============================] - 67s - loss: 0.8477 - categorical_accuracy: 0.8764 - val_loss: 1.2291 - val_categorical_accuracy: 0.7360
Epoch 3/60
437/437 [==============================] - 67s - loss: 0.8396 - categorical_accuracy: 0.8764 - val_loss: 1.2460 - val_categorical_accuracy: 0.7307
Epoch 4/60
437/437 [==============================] - 68s - loss: 0.8278 - categorical_accuracy: 0.8810 - val_loss: 1.2311 - val_categorical_accuracy: 0.7413
Epoch 5/60
437/437 [==============================] - 69s - loss: 0.8484 - categorical_accuracy: 0.8690 - val_loss: 1.2673 - val_categorical_accuracy: 0.7187
Epoch 6/60
437/437 [==============================] - 68s - loss: 0.8327 - categorical_accuracy: 0.8787 - val_loss: 1.2439 - val_categorical_accuracy: 0.7280
Epoch 7/60
437/437 [==============================] - 70s - loss: 0.8610 - categorical_accuracy: 0.8650 - val_loss: 1.2365 - val_categorical_accuracy: 0.7400
Epoch 8/60
437/437 [==============================] - 68s - los

KeyboardInterrupt: 

In [20]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(lr=1e-4, decay=1e-2),
    metrics=['categorical_accuracy']
)

In [21]:
batch_size = 4
model.fit_generator(
    data_generator.flow(train_x, train_y, batch_size=batch_size),
    steps_per_epoch=int(len(train_y) / batch_size), epochs=60,
    callbacks=[MyClallback(model, 'trainable_layers')],
    validation_data=(test_x, test_y)
)

Epoch 1/60
437/437 [==============================] - 91s - loss: 0.8374 - categorical_accuracy: 0.8673 - val_loss: 1.1811 - val_categorical_accuracy: 0.7680
Epoch 2/60
437/437 [==============================] - 85s - loss: 0.7351 - categorical_accuracy: 0.9119 - val_loss: 1.1437 - val_categorical_accuracy: 0.7853
Epoch 3/60
437/437 [==============================] - 87s - loss: 0.6683 - categorical_accuracy: 0.9342 - val_loss: 1.1588 - val_categorical_accuracy: 0.7707
Epoch 4/60
437/437 [==============================] - 87s - loss: 0.6795 - categorical_accuracy: 0.9296 - val_loss: 1.1459 - val_categorical_accuracy: 0.7813
Epoch 5/60
437/437 [==============================] - 87s - loss: 0.6365 - categorical_accuracy: 0.9445 - val_loss: 1.1304 - val_categorical_accuracy: 0.7813
Epoch 6/60
437/437 [==============================] - 88s - loss: 0.6446 - categorical_accuracy: 0.9388 - val_loss: 1.1425 - val_categorical_accuracy: 0.7800
Epoch 7/60
437/437 [==============================] 

KeyboardInterrupt: 

In [22]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(lr=1e-5, decay=1e-5),
    metrics=['categorical_accuracy']
)

In [23]:
batch_size = 4
model.fit_generator(
    data_generator.flow(train_x, train_y, batch_size=batch_size),
    steps_per_epoch=int(len(train_y) / batch_size), epochs=60,
    callbacks=[MyClallback(model, 'low_lr')],
    validation_data=(test_x, test_y)
)

Epoch 1/60
437/437 [==============================] - 91s - loss: 0.5933 - categorical_accuracy: 0.9559 - val_loss: 1.1599 - val_categorical_accuracy: 0.7760
Epoch 2/60
437/437 [==============================] - 87s - loss: 0.6012 - categorical_accuracy: 0.9537 - val_loss: 1.1422 - val_categorical_accuracy: 0.7947
Epoch 3/60
437/437 [==============================] - 87s - loss: 0.5950 - categorical_accuracy: 0.9565 - val_loss: 1.1212 - val_categorical_accuracy: 0.7880
Epoch 4/60
437/437 [==============================] - 88s - loss: 0.5728 - categorical_accuracy: 0.9594 - val_loss: 1.1477 - val_categorical_accuracy: 0.7853
Epoch 5/60
437/437 [==============================] - 88s - loss: 0.5509 - categorical_accuracy: 0.9662 - val_loss: 1.1583 - val_categorical_accuracy: 0.7813
Epoch 6/60
437/437 [==============================] - 88s - loss: 0.5507 - categorical_accuracy: 0.9634 - val_loss: 1.1199 - val_categorical_accuracy: 0.8053
Epoch 7/60
437/437 [==============================] 

KeyboardInterrupt: 

In [24]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(lr=1e-6, decay=1e-3),
    metrics=['categorical_accuracy']
)

In [25]:
batch_size = 4
model.fit_generator(
    data_generator.flow(train_x, train_y, batch_size=batch_size),
    steps_per_epoch=int(len(train_y) / batch_size), epochs=60,
    callbacks=[MyClallback(model, 'low_lr')],
    validation_data=(test_x, test_y)
)

Epoch 1/60
437/437 [==============================] - 89s - loss: 0.3761 - categorical_accuracy: 0.9977 - val_loss: 1.0475 - val_categorical_accuracy: 0.8080
Epoch 2/60
437/437 [==============================] - 85s - loss: 0.3735 - categorical_accuracy: 0.9966 - val_loss: 1.0427 - val_categorical_accuracy: 0.8053
Epoch 3/60
437/437 [==============================] - 85s - loss: 0.3721 - categorical_accuracy: 0.9971 - val_loss: 1.0441 - val_categorical_accuracy: 0.8067
Epoch 4/60
437/437 [==============================] - 85s - loss: 0.3756 - categorical_accuracy: 0.9949 - val_loss: 1.0416 - val_categorical_accuracy: 0.8040
Epoch 5/60
437/437 [==============================] - 86s - loss: 0.3752 - categorical_accuracy: 0.9954 - val_loss: 1.0439 - val_categorical_accuracy: 0.8053
Epoch 6/60
437/437 [==============================] - 86s - loss: 0.3696 - categorical_accuracy: 0.9949 - val_loss: 1.0454 - val_categorical_accuracy: 0.8080
Epoch 7/60
437/437 [==============================] 

KeyboardInterrupt: 